# Program - Read VOCALS-REx GFS SCM forcing data (binary) and save it into a new netCDF file

**Purpose**

Read VOCALS-REx GFS SCM forcing data and save it into a new netCDF file

**Data**

- NCAR/UCAR EOL -  VOCALS: NCEP GFS Single Column Model Forcing Data
  - https://data.eol.ucar.edu/dataset/89.105
- Data access
  - ORDER data for delivery by FTP
- Documentation
  - https://data.eol.ucar.edu/file/download/41B38ABB023/NCEP_GFS_Single_Column_Model_Forcing_Data_for_VOCALS_Rex.pdf

**Author:** Yi-Hsuan Chen (yihsuan@umich.edu)

**Date:** May 2024



# Functions

## read_data

In [211]:
import struct
import numpy as np

####################
####################
####################
def create_return_arrays(num_time_steps, npoint=25, levs=64):

    variable_shapes = {

    #--- 1d variable, (time)
    'date': (num_time_steps),

    #--- 1d variable, (station)
    'station': (npoint),
    'latitude': (npoint),
    'longitude': (npoint),

    #--- 1d variable, (levs/levs+1)
    'sigi': (levs+1),
    'ak5': (levs+1),
    'bk5': (levs+1),
    'sigl': (levs),
        
    #--- 2d variable, (time, station)
    'zsfc': (num_time_steps, npoint),
    'psfc': (num_time_steps, npoint),
    'tsfc': (num_time_steps, npoint),
    'u10': (num_time_steps, npoint),
    'v10': (num_time_steps, npoint),
    't2': (num_time_steps, npoint),
    'q2': (num_time_steps, npoint),
    'hpbl': (num_time_steps, npoint),
        
    #--- 3d variable, (time, station, levels)
    'u': (num_time_steps, npoint, levs),
    'v': (num_time_steps, npoint, levs),
    't': (num_time_steps, npoint, levs),
    'q': (num_time_steps, npoint, levs),
    'p': (num_time_steps, npoint, levs),
    'omega': (num_time_steps, npoint, levs),
    'dtdt': (num_time_steps, npoint, levs),
    'dqdt': (num_time_steps, npoint, levs),
}

    data_arrays = {var: np.zeros(shape) for var, shape in variable_shapes.items()}

    #--- set a new string variable
    #new_string_var = [['' for _ in range(npoint)] for _ in range(num_time_steps)]
    #data_arrays['date_string'] = new_string_var

    # Initialize the 'station' variable as a list of lists (strings)
    #data_arrays['station_string'] = [['' for _ in range(1)] for _ in range(npoint)]
    
    return data_arrays

####################
####################
####################
def read_data(filename, 
              do_print=False):
    with open(filename, 'rb') as file:
        # Read the header
        header_format = '>13i'
        header_size = struct.calcsize(header_format)
        header_data = struct.unpack(header_format, file.read(header_size))
        
        # Unpack the header data
        uu, hour, month, day, year, nsfc, nflx, nvar, levs, npoint, start_hour, end_hour, step_hour = header_data

        if (do_print):
            print(f'Hour: {hour}, Month: {month}, Day: {day}, Year: {year}')
            print(f'Number of surface variables: {nsfc}')
            print(f'Number of flux variables: {nflx}')
            print(f'Number of variables for each sounding: {nvar}')
            print(f'Number of vertical levels: {levs}')
            print(f'Number of station points: {npoint}')
            print(f'Starting forecast hour: {start_hour}')
            print(f'Ending forecast hour: {end_hour}')
            print(f'Forecast output step: {step_hour}')

        #--- create return data array
        data = create_return_arrays(num_time_steps=1, npoint=npoint, levs=levs)
        time_step = 0
        
        # Skip the first two values before reading sigi
        file.seek(8, 1)  # Skip 8 bytes (2 values)
        
        # Read the second record of vertical sounding levels
        sigi = np.fromfile(file, dtype='>f4', count=levs + 1)
        sigl = np.fromfile(file, dtype='>f4', count=levs)
        ak5 = np.fromfile(file, dtype='>f4', count=levs + 1)
        bk5 = np.fromfile(file, dtype='>f4', count=levs + 1)
            
        # Calculate the number of time steps
        num_time_steps = (end_hour - start_hour) // step_hour + 1

        # Loop over station points
        for i in range(npoint):

            # Skip the first two values before reading surface_data
            file.seek(8, 1)  # Skip 8 bytes (2 values)

            # Read surface variables
            surface_vars_format = f'>{nsfc}f'
            #surface_vars_format = f'>50f'
            surface_vars_size = struct.calcsize(surface_vars_format)
            surface_data = struct.unpack(surface_vars_format, file.read(surface_vars_size))

            # Print each element in surface_data
            #print(f'Surface Data for Station {i+1}:')                 
            if (do_print):
                for j, value in enumerate(surface_data):
                    print(f'surface_data {j}: {value}')

            # Skip the first two values before reading flux_data
            file.seek(8, 1)  # Skip 8 bytes (2 values)
            
            # Read flux type variables if nflx > 0
            if nflx > 0:
                flux_vars_format = f'>{nflx}f'
                flux_vars_size = struct.calcsize(flux_vars_format)
                flux_data = struct.unpack(flux_vars_format, file.read(flux_vars_size))

                # Print flux data for debugging purposes
                if (do_print):
                    for j, value in enumerate(flux_data):
                        print(f'flux_data {j}: {value}')
                        
            # Read vertical levels data
            file.seek(8, 1)  # Skip 8 bytes (2 values)  
            u = np.fromfile(file, dtype='>f4', count=levs)

            file.seek(8, 1)  # Skip 8 bytes (2 values)  
            v = np.fromfile(file, dtype='>f4', count=levs)

            file.seek(8, 1)  # Skip 8 bytes (2 values)  
            t = np.fromfile(file, dtype='>f4', count=levs)

            file.seek(8, 1)  # Skip 8 bytes (2 values)  
            q = np.fromfile(file, dtype='>f4', count=levs)

            file.seek(8, 1)  # Skip 8 bytes (2 values)  
            p = np.fromfile(file, dtype='>f4', count=levs)

            if nvar > 5:
                file.seek(8, 1)  # Skip 8 bytes (2 values)
                omega = np.fromfile(file, dtype='>f4', count=levs)

                file.seek(8, 1)  # Skip 8 bytes (2 values)
                dtdt = np.fromfile(file, dtype='>f4', count=levs)

                file.seek(8, 1)  # Skip 8 bytes (2 values)
                dqdt = np.fromfile(file, dtype='>f4', count=levs)

            if nvar > 8:
                file.seek(8, 1)  # Skip 8 bytes (2 values)
                cloud_water = np.fromfile(file, dtype='>f4', count=levs)

                file.seek(8, 1)  # Skip 8 bytes (2 values)
                cloud_water_tend = np.fromfile(file, dtype='>f4', count=levs)

                file.seek(8, 1)  # Skip 8 bytes (2 values)
                cloud_fraction = np.fromfile(file, dtype='>f4', count=levs)

            #--- save variables into data array

            str1 = f"{year}{month}{day:02}{hour:02}"
            data['date'][time_step] = str1
            data['station'][i] = f"{i+1:02}"

            #--- 1d variable 
            data['latitude'][i] = surface_data[0]
            data['longitude'][i] = surface_data[1]
            data['sigi'][:] = sigi
            data['sigl'][:] = sigl
            data['ak5'][:] = ak5
            data['bk5'][:] = bk5
            
            #--- 2d variable, (time, station)
            data['zsfc'][time_step, i] = surface_data[2]
            data['psfc'][time_step, i] = surface_data[3]
            data['tsfc'][time_step, i] = surface_data[4]
            
            data['u10'][time_step, i] = flux_data[17]
            data['v10'][time_step, i] = flux_data[18]
            data['t2'][time_step, i] = flux_data[19]
            data['q2'][time_step, i] = flux_data[20]
            data['hpbl'][time_step, i] = flux_data[26]

            #--- 3d variable, (time, station, levs/levs+1)
            data['u'][time_step, i, :] = u
            data['v'][time_step, i, :] = v
            data['t'][time_step, i, :] = t
            data['q'][time_step, i, :] = q
            data['p'][time_step, i, :] = p
            data['omega'][time_step, i, :] = omega
            data['dtdt'][time_step, i, :] = dtdt
            data['dqdt'][time_step, i, :] = dqdt
            #data[''][time_step, i, :] = 

        if (do_print):
            print('Sigi:', sigi)
            print('Sigl:', sigl)
            print('Ak5:', ak5)
            print('Bk5:', bk5)
            print('u:', u)
            print('v:', v)
            print('t:', t)
            print('q:', q)
            print('omega:', omega*1e+5/86400)
            print('dtdt', dtdt)
            print('dqdt', dtdt)
            print('cloud_water', cloud_water)
            print('cloud_water_tend', cloud_water_tend)
            print('cloud_fraction', cloud_fraction)

    return data

# Example usage
filename = '../original/vocalsgfs.2008101100'

#data = read_data(filename, do_print=True)
data = read_data(filename, do_print=False)

#data['q2']
#data['p'][0,11,:]
#data['latitude']
#print(data['date'].astype(int))
#print(data['station'])

#data

#data_all = create_return_arrays(num_time_steps=2, npoint=25, levs=64)

#filename = '../original/vocalsgfs.2008101100'
#read_data(filename, data_all)

## create_xarray_dataset

In [242]:
import xarray as xr
import numpy as np
from datetime import datetime, timedelta

def create_xarray_dataset(num_time_steps=122, num_station=25, num_levels=64):
    # Define the coordinate arrays
    times = np.arange(num_time_steps)
    stations = np.arange(1, num_station+1)
    levels_mid = np.arange(num_levels)
    levels_int = np.arange(num_levels + 1)

    # Create the xarray Dataset with coordinates
    ds = xr.Dataset(
        coords={
            'time': ('time', times),
            'station': ('station', stations),
            'lev_mid': ('lev_mid', levels_mid),
            'lev_int': ('lev_int', levels_int),
        }
    )

    # Define attributes for coordinates
    coordinate_attributes = {
        'time': {'units': 'hours since 2024-01-01 00:00:00', 'long_name': 'time'},
        'station': {'units': '1', 'long_name': 'station index, VOCALS[##]'},
        'lev_mid': {'units': '1', 'long_name': 'sigma level at midpoint (start from the near surface level)'},
        'lev_int': {'units': '1', 'long_name': 'sigma level at interface (start from the near surface level)'},
    }

    # Assign attributes to coordinates
    for coord_name, attrs in coordinate_attributes.items():
        ds.coords[coord_name].attrs = attrs

    # Define the dimension tuples
    time_only = ('time',)
    station_only = ('station',)
    lev_mid_only = ('lev_mid',)
    lev_int_only = ('lev_int',)
    time_station = ('time', 'station')
    time_station_levmid = ('time', 'station', 'lev_mid')
    time_station_levint = ('time', 'station', 'lev_int')

    # Define the data variables and their dimensions
    variable_specs = {
        'date': time_only,
        'latitude': station_only,
        'longitude': station_only,
        'sigl': lev_mid_only,
        'sigi': lev_int_only,
        'ak5': lev_int_only,
        'bk5': lev_int_only,
        'zsfc': time_station,
        'psfc': time_station,
        'tsfc': time_station,
        'u10': time_station,
        'v10': time_station,
        't2': time_station,
        'q2': time_station,
        'hpbl': time_station,
        'u': time_station_levmid,
        'v': time_station_levmid,
        't': time_station_levmid,
        'q': time_station_levmid,
        'p': time_station_levmid,
        'omega': time_station_levmid,
        'dtdt': time_station_levmid,
        'dqdt': time_station_levmid,
    }

    # Define attributes for each variable
    variable_attributes = {
        'date': {'units': 'none', 'long_name': 'YYYYMMDDHH (UTC)'},
        'latitude': {'units': 'degrees_north', 'long_name': 'latitude of station'},
        'longitude': {'units': 'degrees_east', 'long_name': 'longitude of station'},
        'zsfc': {'units': 'm', 'long_name': 'model surface height for the station'},
        'psfc': {'units': 'Pa', 'long_name': 'model surface pressure for the station'},
        'tsfc': {'units': 'K', 'long_name': 'model surface temperature'},
        'u10': {'units': 'm/s', 'long_name': 'model derived 10-meter zonal wind'},
        'v10': {'units': 'm/s', 'long_name': 'model derived 10-meter meridional wind'},
        't2': {'units': 'K', 'long_name': 'model derived 2-meter temperature'},
        'q2': {'units': 'kg/kg', 'long_name': 'model derived 2-meter specific humidity'},
        'hpbl': {'units': 'm', 'long_name': 'model diagnosed planetary boundary layer depth'},
        'u': {'units': 'm/s', 'long_name': 'model zonal wind velocity'},
        'v': {'units': 'm/s', 'long_name': 'model meridional wind velocity'},
        't': {'units': 'K', 'long_name': 'model temperature'},
        'q': {'units': 'kg/kg', 'long_name': 'model specific humidity'},
        'p': {'units': 'Pa', 'long_name': 'model pressure'},
        'omega': {'units': 'Pa/s', 'long_name': 'model derived omega'},
        'dtdt': {'units': 'K/s', 'long_name': 'model derived dtdt (advection)'},
        'dqdt': {'units': 'kg/kg/s', 'long_name': 'model derived dqdt (advection)'},
        'sigl': {'units': '1', 'long_name': 'model sigma level at midpoint (count upward)'},
        'sigi': {'units': '1', 'long_name': 'model sigma level at interface (count upward)'},
        'ak5': {'units': '1', 'long_name': 'A-coefficient'},
        'bk5': {'units': '1', 'long_name': 'B-coefficient'},
    }

    
    # Add the data variables to the Dataset
    for var_name, dims in variable_specs.items():
        shape = tuple(ds.coords[dim].size for dim in dims)
        ds[var_name] = xr.DataArray(np.random.rand(*shape), dims=dims)
        ds[var_name].attrs = variable_attributes.get(var_name, {})

    # Define global attributes
    global_attributes = {
        'title': 'NCAP GFS Single Column Model Forcing Data for VOCALS-Rex',
        'data_source': 'NCAR/UCAR EOL - VOCALS: NCEP GFS Single Column Model Forcing Data, https://data.eol.ucar.edu/dataset/89.105',
        'data_reference': 'https://data.eol.ucar.edu/file/download/41B38ABB023/NCEP_GFS_Single_Column_Model_Forcing_Data_for_VOCALS_Rex.pdf',
        'history': f'Created on {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}',
        'author': 'Yi-Hsuan Chen (yihsuanc@gate.sinica.edu.tw)'
    }

    # Assign global attributes to the Dataset
    ds.attrs = global_attributes
    
    return ds

# Example usage
ds = create_xarray_dataset()
#ds


## process_files_in_directory

In [243]:
import os
def process_files_in_directory(directory='/lfs/home/yihsuanc/data/data.TaiESM1_scm/iop/VOCALS-REx/original/'):

    file_names = []
    for filename in os.listdir(directory):
        if filename.startswith('vocalsgfs'):
            #print(filename)
            file_path = os.path.join(directory, filename)
            file_names.append(file_path)
    return sorted(file_names)

#file_names = process_files_in_directory()
#for ff in file_names:
#    print(ff)

## read_data_all_to_ds

In [290]:
def read_data_all_to_ds():
    #--- get all vocalsgfs file paths
    file_names = process_files_in_directory()
    
    #--- Create an xarray dataset
    ds_all = create_xarray_dataset()
    
    #--- process file_names  and then save into ds_all
    for i, ff in enumerate(file_names):
        #print(f'Read [{i}, {ff}]')
        data = read_data(ff)    
    
        for var_name in data.keys():
            if var_name in ds_all.variables:
                ndim = data[var_name].ndim
                #print(f"Variable: {var_name}, ndim: {ndim}")
        
                if (ndim == 1 and var_name == 'date'):
                   ds_all[var_name][i] = data[var_name][0]
                if (ndim == 2):
                   ds_all[var_name][i,:] = data[var_name][0,:]
                elif (ndim == 3):
                   ds_all[var_name][i,:,:] = data[var_name][0,:,:]
    
        var_1d = ['latitude', 'longitude', 'sigl', 'sigi', 'ak5', 'bk5']
        for var_name in var_1d:
            ds_all[var_name][:] = data[var_name][:]
    
    #--- change coordinate
    ds_all = ds_all.assign_coords(time=ds_all['date'].values)
    ds_all.coords['time'].attrs = ds_all['date'].attrs
    
    ds_all = ds_all.assign_coords(lev_mid=ds_all['sigl'][:].values)
    ds_all.coords['lev_mid'].attrs = ds_all['sigl'].attrs
    
    ds_all = ds_all.assign_coords(lev_int=ds_all['sigi'][:].values)
    ds_all.coords['lev_int'].attrs = ds_all['sigi'].attrs

    return ds_all

# Examples 

## Read a single date, all 25 stations

In [291]:
filename = '../original/vocalsgfs.2008100100'
data1 = read_data(filename, do_print=False)

variable_dimensions = {var: arr.shape for var, arr in data1.items()}
print(variable_dimensions)

{'date': (1,), 'station': (25,), 'latitude': (25,), 'longitude': (25,), 'sigi': (65,), 'ak5': (65,), 'bk5': (65,), 'sigl': (64,), 'zsfc': (1, 25), 'psfc': (1, 25), 'tsfc': (1, 25), 'u10': (1, 25), 'v10': (1, 25), 't2': (1, 25), 'q2': (1, 25), 'hpbl': (1, 25), 'u': (1, 25, 64), 'v': (1, 25, 64), 't': (1, 25, 64), 'q': (1, 25, 64), 'p': (1, 25, 64), 'omega': (1, 25, 64), 'dtdt': (1, 25, 64), 'dqdt': (1, 25, 64)}


## Read all dates, all 25 stations

In [292]:
ds_all = read_data_all_to_ds()
print(ds_all)

<xarray.Dataset>
Dimensions:    (time: 122, station: 25, lev_mid: 64, lev_int: 65)
Coordinates:
  * time       (time) float64 2.008e+09 2.008e+09 ... 2.008e+09 2.008e+09
  * station    (station) int64 1 2 3 4 5 6 7 8 9 ... 17 18 19 20 21 22 23 24 25
  * lev_mid    (lev_mid) float64 0.9973 0.9917 0.9852 ... 0.00101 0.0003212
  * lev_int    (lev_int) float64 1.0 0.9947 0.9886 ... 0.001378 0.0006425 0.0
Data variables: (12/23)
    date       (time) float64 2.008e+09 2.008e+09 ... 2.008e+09 2.008e+09
    latitude   (station) float64 -20.0 -20.0 -20.0 -20.0 ... -28.0 -30.0 -23.5
    longitude  (station) float64 -95.0 -92.5 -90.0 -87.25 ... -85.0 -85.0 -70.0
    sigl       (lev_mid) float64 0.9973 0.9917 0.9852 ... 0.00101 0.0003212
    sigi       (lev_int) float64 1.0 0.9947 0.9886 ... 0.001378 0.0006425 0.0
    ak5        (lev_int) float64 0.0 0.06425 0.1378 0.222 ... 0.000575 0.0 0.0
    ...         ...
    t          (time, station, lev_mid) float64 292.8 292.4 ... 264.2 236.2
    q     

## Check combined dataset with inividual one

In [297]:
date = 2008101612
station = 11

#--- combined dataset
ds_all.sel(time=date, station=station)

#--- individual data array
filename = f'../original/vocalsgfs.{date}'
data2 = read_data(filename, do_print=False)

#--- check lat & lon
var_1d = ['latitude', 'longitude']
for var_name in var_1d:
    print('ds_all: ', ds_all[var_name].sel(station=station).values)
    print('single: ', data2[var_name][station-1])

#--- check 2d variable (time, station)
var_2d = ['psfc','tsfc','t2','q2']
for var_name in var_2d:
    print(var_name)

    vv1 = ds_all[var_name].sel(time=date, station=station)
    vv2 = data2[var_name][0, station-1]
    vv1m2 = vv1 - vv2
    print('  ds_all: ', vv1.values)
    print('  ds_all - single: ', vv1m2.values)
    #print('ds_all: ', ds_all[var_name].sel(time=date, station=16).values)
    #print('single: ', data2[var_name][0, station-1, :])

#--- check 3d variable (time, station, levs)
var_nd = ['t','u','v','q','p']
for var_name in var_nd:
    print(var_name)

    vv1 = ds_all[var_name].sel(time=date, station=station)
    vv2 = data2[var_name][0, station-1, :]
    vv1m2 = vv1 - vv2
    print('  ds_all: ', vv1.values)
    print('  ds_all - single: ', vv1m2.values)


ds_all:  -20.0
single:  -20.0
ds_all:  -70.0
single:  -70.0
psfc
  ds_all:  92821584.0
  ds_all - single:  0.0
tsfc
  ds_all:  0.0005159290740266442
  ds_all - single:  0.0
t2
  ds_all:  291.2839050292969
  ds_all - single:  0.0
q2
  ds_all:  0.007325334474444389
  ds_all - single:  0.0
t
  ds_all:  [290.35858154 289.87283325 289.51748657 289.22332764 289.07583618
 289.16253662 289.28857422 289.33886719 289.365448   289.30908203
 289.15063477 288.85018921 288.3890686  287.75067139 286.92098999
 285.86489868 284.73895264 283.41021729 281.84939575 279.86102295
 277.28851318 273.97070312 270.33227539 266.82763672 263.58724976
 260.27078247 257.50271606 254.74697876 250.80262756 244.98513794
 239.7756958  233.56271362 228.3825531  222.5932312  216.68540955
 211.48352051 205.36715698 201.81356812 196.72789001 195.54672241
 194.63632202 197.384552   201.27037048 202.88566589 205.52885437
 209.38372803 211.01477051 211.53338623 211.49095154 213.50582886
 216.48968506 218.15408325 221.49659729